
<h1>Data requirements :</h1>

->Geographical coordinates(latitide and longitude) to find our where exactly it is located.

->Population of the neighborhood where the restaurant is located.

->Average income of neighborhood to know how much is the restaurant worth.

To access location of a restaurant, its Latitude and Longitude is to be known so that we can point at its coordinates and create a map displaying all the restaurants with its labels respectively.<br>
Population of a neighborhood is very important factor in determining a restaurant's growth and amount of customers who turn up to eat.<br> Logically, the more the population of a neighborhood, the more people will be interested to walk openly into a restaurant and less the population, less number of people frequently visit a restaurant. Also if more people visit, better the restaurant is rated because it is accessed by different people with different taste. Hence is is very important factor.
<br>Income of a neighborhood is also very important factor as population was. Income is directly proportional to richness of a neighborhood. If people in a neighborhood earns more than an average income, then it is very much possible that they will spend more however not always true with very less probability. <br>So an restaurant accessment is proportional to income of a neighborhood.

<h3> Initial Data </h3>
Taken the longitutes and latitudes from the gogle map and prepared the dataset.
The data set contains Place with its Latitudes and Longitutes.
The Population in that area and the average income of the people surrunding the area.

In [1]:
df=pd.read_csv("Data/Part1.csv",index_col=0)

In [2]:
df.head()

,Neighborhoods,Latitude,Longitude,Population,City,AverageIncome
Borough,,,,,,
1,Adyar Bus Debot.,"12°59'50"" N","80°15'25"" E",14212,Chennai,285833
1,Adyar Signal,"13°00'23"" N","80°15'27"" E",32019,Chennai,503997
4,Alandur,"13°00'28"" N","80°12'35"" E",49787,Chennai,578503
2,Ambattur,"13°06'36"" N","80°10'12"" E",48309,Chennai,267914
2,Anna Arch,"13°04'28"" N","80°13'06"" E",23188,Chennai,153030


<h3> Foursquare API to explore the neighborhoods and segment them.</h3>

In [4]:
CLIENT_ID = 'CQXS3MAKCWX0DIMA1SBERK50VYMI3OXMAVDMGW4XVI5ZAW4N' # your Foursquare ID
CLIENT_SECRET = 'VSH03ZX0SPCCBUXCS1350EWHAAKZDX2ITCQ1KRCVU1GB0534' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CQXS3MAKCWX0DIMA1SBERK50VYMI3OXMAVDMGW4XVI5ZAW4N
CLIENT_SECRET:VSH03ZX0SPCCBUXCS1350EWHAAKZDX2ITCQ1KRCVU1GB0534


In [12]:
url="https://api.foursquare.com/v2/venues/search/72057594039192463/client_id="+CLIENT_ID+"/client_secret="+CLIENT_SECRET+"/v=20180605"

In [13]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 400,
  'errorType': 'invalid_auth',
  'errorDetail': 'Missing access credentials. See https://developer.foursquare.com/docs/api/configuration/authentication for details.',
  'requestId': '5e2c569e618f43001b0d4a03'},
 'response': {}}

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )